In [1]:
import pandas as pd
import numpy as np
import torch
import os

In [2]:
if os.path.isfile('./data.pt'):
    data = torch.load('data.pt')
else:
    data = pd.read_excel('220106_자료/KAMIR-V 1Y FU DATA_Cumulative Death_20220106 변수 정리.xlsx', sheet_name='KAMIR-V 1년 F-U DATA', skiprows=[0])

In [3]:
from utils import get_col_dict

col_dict = get_col_dict()

In [4]:
numeric_cols = [4, 28, 31, 32, 33, 36, 37, 170, 174, 175, 177, 178, 259, 262, 265, 268, 271, 274, 277, 280, 283, 286, 289, 292, 293, 294, 297, 300, 303, 306, 309, 312, 313, 316, 317, 327, 328, 329, 369, 372, 375, 379, 396, 401, 402, 403, 438, 439, 440, 474, 476, 478, 479, 481, 482, 489, 491, 493, 495, 497, 500, 503, 506, 509, 512, 515, 518, 521, 524, 527, 530, 532, 535, 537, 540, 542, 545, 547, 550, 552, 555, 557, 560, 562, 565, 567, 570, 572, 575, 577, 580, 582, 585, 587, 590, 592, 595, 597, 600, 602, 605, 607, 610, 612]
numeric_data = data.iloc[:, numeric_cols]

[170, 174, 175, 177, 178]은 백분율로 변환 필요

259 - peak Troponin T값이 어떤 건지 ex) 백분율, 실수값
262 - 최초의 clopidogrel resistance 값
265 - 최초의 aspirin resistance 값
268 - HbA1c값
271 - BNP값
274 - NT-proBNP 값
277 - hsCRP 값
280 - LDL-c 값
283 - HDL-c 값
286 - TG 값
289 - TC 값
292 - WBC 값
293 - neutrophil 값 (WBC 시행일과 같음)
294 - lymphocyte 값(WBC 시행일과 같음)
297 - 최대 Troponin I 값
300 - 최대 CK-MB 값 
303 - 최대 CK 값 
306 - 최대 cr 값 
309 - 최초의 cr 값 
312 - 최초의 glucose 값 
313 - 최초의 PLT 값 
316 - 최저의 Hb 값 
317 - 입원 당시 Hb 값
369 - BB  용량 (변수 다시 정해서 qartile 나누어서 분석에 이용하기)
      Beta-blocker 용량 quartile 1~4 (비어 있는 경우는 의미 있는 beta-blocker 아니므로 분석하지 않거나 잘못 기입된 항목으로, 제함)
372 - ACEi  용량(변수 다시 정해서 qartile 나누어서 분석에 이용하기)
375 - ARB  용량(변수 다시 정해서 qartile 나누어서 분석에 이용하기)
379 - statin 용량 (변수 다시 정해서 low intensity, moderate intensity, high intensity 로 나누어 분석에 이용하기)
396 - omega 3 용량

474 - 170과 비교 필요
476 - 172와 비교 필요
478 - 174와 비교 필요
479 - 175와 비교 필요
481 - 177과 비교 필요
482 - 178과 비교 필요

489 - 12개월 fu WBC 값
491 - 12개월 fu neutrophil 값
493 - 12개월 fu lymphocyte 값
495 - 12개월 fu Hb 값  (WBC 시행일과 같음)
497 - 12개월 fu platelet 값  (WBC 시행일과 같음)
500 - 12개월 fu glucose 값
503 - 12개월 fu cr 값
506 - 12개월 fu TC 값
509 - 12개월 fu TG 값
512 - 12개월 fu HDL 값
515 - 12개월 fu LDL 값
518 - 12개월 fu hsCRP 값
521 - 12개월 fu NT-proBNP 값
524 - 12개월 fu BNP 값
527 - 12개월 fu HbA1c 값

530 - 12개월 ARB 용량 (on-going 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기, Q46-1(375) 입원 중 투약한 용량과 비교하여 감량/동량/증량 으로 구분하는 변수 만들어 분석에 이용하기)
532 - 12개월 ARB 용량 (new start 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기)
535 - 12개월 omega 3 용량 (on-going 인 경우) 
537 - 12개월 omega 3 용량 (new start 인 경우)

540 - 12개월1st OHA 용량 (on-going 인 경우)  (이 중에서 붉은 색 cell 로 표시한 것을 포함한 group 과 그렇지 않은 group 으로 구분 필요함)
542 - 12개월 1stOHA 용량 (new start 인 경우) (이 중에서 붉은 색 cell 로 표시한 것을 포함한 group 과 그렇지 않은 group 으로 구분 필요함)

545 - 12개월 2nd OHA 처방 여부 : on-going (지속 처방중), stop (쓰다가 중단한 상태), no use (한번도 쓰지 않았음), new start (12개월째 새로 시작됨), unknown 
547 - 12개월 2nd OHA 용량 (new start 인 경우) (이 중에서 붉은 색 cell 로 표시한 것을 포함한 group 과 그렇지 않은 group 으로 구분 필요함)

550 - 12개월 3rd OHA 용량 (on-going 인 경우) (이 중에서 붉은 색 cell 로 표시한 것을 포함한 group 과 그렇지 않은 group 으로 구분 필요함) 
552 - 12개월 3rd OHA 용량 (new start 인 경우) (이 중에서 붉은 색 cell 로 표시한 것을 포함한 group 과 그렇지 않은 group 으로 구분 필요함)

555 - 12개월 anticoagulant 용량 (on-going 인 경우) 
557 - 12개월 anticoagulant 용량 (new start 인 경우)

560 - 12개월 ezetimibe 용량 (on-going 인 경우) 
562 - 12개월 ezetimibe 용량 (new start 인 경우)

565 - 12개월 Fibrate 용량 (on-going 인 경우) 
567 - 12개월 Fibrate 용량 (new start 인 경우)

570 - 12개월 Statin 용량 (on-going 인 경우)  (변수 다시 정해서 low intensity, moderate intensity, high intensity 로 나누어 분석에 이용하기)
572 - 12개월 Statin 용량 (new start 인 경우)  (변수 다시 정해서 low intensity, moderate intensity, high intensity 로 나누어 분석에 이용하기)

575 - 12개월 Aspirin 용량 (on-going 인 경우) 
577 - 12개월 Aspirin 용량 (new start 인 경우)

580 - 12개월 ACE inhibitor 용량 (on-going 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기, Q45-1(372) 입원 중 투약한 용량과 비교하여 감량/동량/증량 으로 구분하는 변수 만들어 분석에 이용하기)
582 - 12개월 ACE inhibitor 용량 (new start 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기)

585 - 12개월 Beta-blocker 용량 (on-going 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기, Q44-1(369) 입원 중 투약한 용량과 비교하여 감량/동량/증량 으로 구분하는 변수 만들어 분석에 이용하기)
587 - 12개월 Beta-blocker 용량 (new start 인 경우) : (qartile 나누어서 새로 변수 만들어 분석에 이용하기)

590 - 12개월 Ca-channel blocker 용량 (on-going 인 경우) 
592 - 12개월 Ca-channel blocker 용량 (new start 인 경우)

595 - 12개월 Ticagrelor용량 (on-going 인 경우) 
597 - 12개월 Ticagrelor 용량 (new start 인 경우)

600 - 12개월 Cilostazol용량 (on-going 인 경우) 
602 - 12개월 Cilostazol 용량 (new start 인 경우)

605 - 12개월 Prasugurel용량 (on-going 인 경우) 
607 - 12개월 Prasugurel 용량 (new start 인 경우)

610 - 12개월 Clopidogrel용량 (on-going 인 경우) 
612 - 12개월 Clopidogrel 용량 (new start 인 경우)

369 - BB  용량 (변수 다시 정해서 qartile 나누어서 분석에 이용하기)
      Beta-blocker 용량 quartile 1~4 (비어 있는 경우는 의미 있는 beta-blocker 아니므로 분석하지 않거나 잘못 기입된 항목으로, 제함)

In [5]:
print("복부둘레 측정값 존재 비율 =", numeric_data['A.C.1'].isna().sum() / len(numeric_data['A.C.1']))

복부둘레 측정값 존재 비율 = 0.7626695674430509


In [6]:
for c in numeric_data.columns:
    print(col_dict[c])

나이
복부둘레 측정값
수축기혈압
이완기 혈압
맥박
키
몸무게
좌심실 구혈률
좌심실 수축기말 직경 (소수점 자리 빼고)
좌심실 이완기말 직경 (소수점 자리 빼고)
좌심실 수축기말 용적 (소수점 자리 빼고)
좌심실 이완기말 용적 (소수점 자리 빼고)
peak Troponin T 값
최초의 clopidogrel resistance 값
최초의 aspirin resistance 값
Hba1c값
BNP 값
NT-proBNP 값
hsCRP 값
LDL-c 값
HDL-c 값
TG 값
TC 값 
WBC 값
neutrophil 값 (WBC 시행일과 같음)
lymphocyte 값(WBC 시행일과 같음)
최대 Troponin I 값
최대 CK-MB 값 
최대 CK 값 
최대 cr 값 
최초의 cr 값 
최초의 glucose 값 
최초의 PLT 값 
최저의 Hb 값 
입원 당시 Hb 값 
퇴원 시 수축기 혈압
퇴원 시 이완기 혈압
퇴원 시 맥박
BB  용량 (변수 다시 정해서 qartile 나누어서 분석에 이용하기)
ACEi  용량(변수 다시 정해서 qartile 나누어서 분석에 이용하기)
ARB  용량(변수 다시 정해서 qartile 나누어서 분석에 이용하기)
statin 용량 (변수 다시 정해서 low intensity, moderate intensity, high intensity 로 나누어 분석에 이용하기)
omega 3 용량
6개월째 수축기 혈압
6개월째 이완기 혈압
6개월째 맥박
12개월째 수축기 혈압
12개월째 이완기 혈압
12개월째 맥박
12개월 LVEF 
12개월 MR grade 
12개월 LVESD
12개월 LVEDD
12개월 LVESV 
12개월 LVEDV
12개월 fu WBC 값
12개월 fu neutrophil 값
12개월 fu lymphocyte 값
12개월 fu Hb 값  (WBC 시행일과 같음)
12개월 fu platelet 값  (WBC 시행일과 같음)
12개월 fu glucose 값
12개월 fu cr 값
12개월 fu TC 값
12개월 fu TG 값
